# Closing the Loop on MNIST with Convolutional Networks 

Now that we have a better understanding of how to build networks that effectively analyze images, we’ll revisit the MNIST challenge we’ve tackled over the past several chapters. Here, we’ll use a convolutional network to learn how to recognize handwritten digits. Our feed-forward network was able to achieve a 98.2% accuracy. Our goal will be to push the envelope on this result. 

To tackle this challenge, we’ll build a convolutional network with a pretty standard architecture: two pooling and two convolutional interleaved, followed by a fully connected layer (with dropout, p = 0.5) and a terminal softmax. To make building the network easy, we write a couple of helper methods in addition to our layer generator from the feed-forward network:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('data',one_hot=True)

In [ ]:
def conv2d(input,weight_shape,bias_shape):
    in_ = weight_shape[0]*weight_shape[1]*weight_shape[2]
    weight_init=tf.random_normal_initializer(stddev=(2.8/in_)**0.5)
    W=tf.get_variable("W",weight_shape,initializer=weight_init)
    bias_init=tf.constant_initializer(value=0)
    b=tf.get_variable("b",bias_shape,initializer=bias_init)
    conv_out=tf.nn.conv2d(input,W,strides=[1,1,1,1],padding='SAME')
    return tf.nn.relu(tf.nn.bias_add(conv_out,b))

def max_pool(input,k=2):
    return tf.nn.max_pool(input,ksize=[1,k,k,1],
                          strides=[1,k,k,1],padding='SAME')

def layer(input, weight_shape, bias_shape):
    weight_stddev = (2.0/weight_shape[0])**0.5 
    w_init = tf.random_normal_initializer(stddev=weight_stddev)   
    bias_init = tf.constant_initializer(value=0) 
    W = tf.get_variable("W", weight_shape, 
                        initializer=w_init)  
    b = tf.get_variable("b", bias_shape, 
                        initializer=bias_init)
    return tf.nn.relu(tf.matmul(input, W) + b)

The first helper method generates a convolutional layer with a particular shape. We set the stride to be to be 1 and the padding to keep the width and height constant between input and output tensors. We also initialize the weights using the same heuristic we used in the feed-forward network. In this case, however, the number of incoming weights into a neuron spans the filter’s height and width and the input tensor’s depth.

The second helper method generates a max pooling layer with non-overlapping windows of size k. The default, as recommended, is k=2, and we’ll use this default in our MNIST convolutional network. 

With these helper methods, we can now build a new inference constructor:

In [ ]:
def inference(x,keep_prob):
    x=tf.reshape(x,shape=[-1,28,28,1])
    with tf.variable_scope("conv_1"):
        conv_1=conv2d(x,[5,5,1,32],[32])
        pool_1=max_pool(conv_1)
    with tf.variable_scope("conv_2"):
        conv_2=conv2d(pool_1,[5,5,32,64],[64])
        pool_2=max_pool(conv_2)
    with tf.variable_scope("fc"):
        pool_2_flat=tf.reshape(pool_2,[-1,7*7*64])
        fc_1=layer(pool_2_flat,[7*7*64,1024],[1024])
        
        #apply dropout
        fc_1_drop=tf.nn.dropout(fc_1,keep_prob=keep_prob)
        
    with tf.variable_scope("output"):
        output=layer(fc_1_drop,[1024,10],[10])
    return output

def evaluate(output, y):  
    correct_prediction = tf.equal(tf.argmax(output, 1),
                                  tf.argmax(y, 1))   
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 
                                      tf.float32))
    return accuracy

The code  here is quite easy to follow. We first take the flattened versions of the input pixel values and reshape them into a tensor of the N ×28×28×1, where N is the number of examples in a minibatch, 28 is the width and height of each image, and 1 is the depth (because the images are black and white; if the images were in RGB color, the depth would instead be 3 to represent each color map). We then build a convolutional layer with 32 filters that have spatial extent 5. This results in taking an input volume of depth 1 and emitting a output tensor of depth 32. This is then passed through a max pooling layer which compresses the information. We then build a second convolutional layer with 64 filters, again with spatial extent 5, taking an input tensor of depth 32 and emitting an output tensor of depth 64. This, again, is passed through a max pooling layer to compress information.

We then prepare to pass the output of the max pooling layer into a fully connected layer. To do this, we flatten the tensor. We can do this by  computing the full size of each “subtensor” in the minibatch. We have 64 filters, which corresponds to the depth of 64. We now have to determine the height and width after passing through two max pooling layers. Using the formulas we found in the previous section, it’s easy to confirm that each feature map has a height and width of 7. Confirming this is left as an exercise for the reader.

After the reshaping operation, we use a fully connected layer to compress the flattened representation into a hidden state of size 1,024. We use a dropout probability in this layer of 0.5 during training and 1 during model evaluation (standard procedure for employing dropout). Finally, we send this hidden state into a softmax output layer with 10 bins (the softmax is, as usual, performed in the loss constructor for better performance). 

Finally, we train our network using the Adam optimizer. After several epochs over the dataset, we achieve an accuracy of 99.4%, which isn’t state of the art (approximately 99.7 to 99.8%), but is very respectable.

In [ ]:
def loss(output, y): 
    dot_product = y * tf.log(output)
    # Reduction along axis 0 collapses each column into a  
    # single value, whereas reduction along axis 1 collapses 
    # each row into a single value. In general, reduction along    
    # axis i collapses the ith dimension of a tensor to size 1. 
    xentropy = -tf.reduce_sum(dot_product, reduction_indices=1)   
    loss = tf.reduce_mean(xentropy)
    return loss 

def training(cost, global_step):  
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step) 
    return train_op 

In [ ]:
learning_rate=0.001
training_epochs=1000
batch_size=100
display_step=1

In [ ]:
with tf.Graph().as_default():
    x=tf.placeholder("float",[None,784])
    y=tf.placeholder("float",[None,10])
    
    output=inference(x,True)
    cost=loss(output,y)
    global_step=tf.Variable(0,name="global_step",trainable=False)
    train_op=training(cost,global_step)
    eval_op=evaluate(output,y)
    saver=tf.train.Saver()
    sess=tf.Session()
    
    init_op=tf.initialize_all_variables()
    sess.run(init_op)
    
    for epoch in range(training_epochs):
        avg_cost=0
        total_batch=int(mnist.train.num_examples/batch_size)
        #loop over all batchs
        for i in range(total_batch):
            mbatch_x,mbatch_y=mnist.train.next_batch(batch_size=batch_size)            
            feed_dict={x:mbatch_x,y:mbatch_y}
            sess.run(train_op,feed_dict=feed_dict)
            minibatch_cost=sess.run(cost,feed_dict=feed_dict)
            avg_cost+=minibatch_cost/total_batch
            
            if epoch%display_step==0:
                val_feed_dict= {
                    x:mnist.validation.images,
                    y:mnist.validation.labels
                }
                accuracy=sess.run(eval_op,feed_dict=val_feed_dict)
                print("Validation error: ",(1-accuracy))